# Project Plan your trip with Kayak ✈️

Summary :

- Scrape data from destinations
- Get weather data from each destination
- Get hotels' info about each destination
- Store all the information above in a data lake
- Extract, transform and load cleaned data from your datalake to a data warehouse
- MAPBOX

In [1]:
import pandas as pd
import json
import requests
from pprint import pprint

## Scrape data from destinations

In [2]:
# Let's open our json with the name of all the cities
with open("response_cities.json", "r") as f:
    cities =  json.loads(f.read())

# Create a list containing our cities names
list_of_cities = [[city for key, city in ele.items()] for ele in cities]

# Show the first name of the list
list_of_cities[0]

['Mont Saint Michel']

In [3]:
# Create a list containing the response from the coordinates api
list_api = [requests.get(f"https://nominatim.openstreetmap.org/search/{city}?format=json&addressdetails=1&limit=1").json() for city in list_of_cities]

In [4]:
# Now, create a new list containing the coordinates
list_cities_infos = []
for i in range(len(list_of_cities)):
    res = dict((val, list_api[i][0][val]) for val in ['lat', 'lon'] # select latitude, longitude keys to acces to values
           if val in list_api[i][0])
    i = i + 1
    list_cities_infos.append(res)

In [5]:
# Create two datasets : the first will contain names of cities, the second the coordinates
df_cities = pd.DataFrame(list_of_cities, columns=['city'])
df1 = pd.DataFrame(list_cities_infos)

# Merge ours datasets 
df = df_cities.join(df1)

# Display the first 5 rows
df.head()

,city,lat,lon
0,Mont Saint Michel,48.6359541,-1.511459954959514
1,St Malo,48.649518,-2.0260409
2,Bayeux,49.2764624,-0.7024738
3,Le Havre,49.4938975,0.1079732
4,Rouen,49.4404591,1.0939658


## Get weather data from each destination

In [6]:
# Create a list containing the dataset's values
list_values = df.values.tolist()

# Instanciate the key 
key = "********"

# Create a new list containing our request from the weather api
list_openweather = [
    requests.get(f"https://api.openweathermap.org/data/2.5/forecast?lat={list_values[i][1]}&lon={list_values[i][2]}&exclude=daily&units=metric&appid={key}").json()
    for i in range(len(list_values))]

# This is our final list containing the name of the city with its own weather informations  
list_cities_weather = [[[city, list_openweather[i]] for city in list_of_cities] for i in range(len(list_values))]

In [7]:
# Create the average fonction
def avg(lst):
    return sum(lst)/len(lst)

# Create a new list containing temperature and humidity
list_weather = []

# Now, iterate in weather list to get temperature and humidity
for i in range(len(list_of_cities)):
    li = list_cities_weather[i][0][1]['list'] # this is the starter path to get our values
    temp = round(avg([li[j]['main']['temp'] for j in range(len(li))])) # get the temperature, don't forget to round it
    hum = round(avg([li[j]['main']['humidity'] for j in range(len(li))])) # do the same to get humidity
    list_temp_hum = [temp, hum] # create a list containing each city values
    list_weather.append(list_temp_hum)

# Show the first 5 values
list_weather[:5]

[[4, 85], [5, 80], [5, 81], [5, 80], [2, 87]]

In [8]:
# Create a new dataset with temperature and humidity informations
weather = pd.DataFrame(list_weather, columns=['temperature', 'humidity'])
# Now, merge our datasets in order to have the name of the cities with their own temperature and humidity informations
df = df_cities.join(weather)
df.head()

,city,temperature,humidity
0,Mont Saint Michel,4,85
1,St Malo,5,80
2,Bayeux,5,81
3,Le Havre,5,80
4,Rouen,2,87


## Get hotels' info about each destination

In [9]:
# Import hotels.json and put it into a new hotels dataset
df_hotels = pd.read_json('hotels.json')
df_hotels.head()

,city,hotel_name,score,description,url,gps
0,Le Havre,Duplex lumineux tout confort,8.0,Duplex lumineux tout confort in Le Havre offer...,https://www.booking.com/hotel/fr/duplex-lumine...,"49.48627610,0.13162500"
1,Le Havre,Les gîtes Havrais Bis,7.1,Located within 1.2 km of Le Havre Beach and 1....,https://www.booking.com/hotel/fr/les-gites-san...,"49.50439760,0.10028140"
2,Bayeux,La Closerie Teranga,9.1,Managed by a private host,https://www.booking.com/hotel/fr/la-closerie-t...,"49.27804172,-0.70187662"
3,Amiens,WHITE HOUSE DHAVERNAS,8.6,Managed by a private host,https://www.booking.com/hotel/fr/white-house-d...,"49.88146960,2.29987140"
4,Le Havre,Le Petit Vatel,7.1,"Le Petit Vatel is located in l’espace Perret, ...",https://www.booking.com/hotel/fr/petitvatel.en...,"49.49111700,0.10451764"


In [10]:
# Create a fonction to get latitude
def get_latitude(gps):
    return float(gps.split(', ')[0].split(',')[0])
# Create a fonction to get longitude
def get_longitude(gps):
    return float(gps.split(', ')[0].split(',')[1])

In [11]:
# Apply ours fonctions in order to get two new columns : latitude and longitude
df_hotels['latitude'] = df_hotels['gps'].apply(get_latitude)
df_hotels['longitude'] = df_hotels['gps'].apply(get_longitude)

# Now drop gps and url columns
df_hotels = df_hotels.drop(['gps', 'url'], axis=1)
df_hotels.head()

,city,hotel_name,score,description,latitude,longitude
0,Le Havre,Duplex lumineux tout confort,8.0,Duplex lumineux tout confort in Le Havre offer...,49.486276,0.131625
1,Le Havre,Les gîtes Havrais Bis,7.1,Located within 1.2 km of Le Havre Beach and 1....,49.504398,0.100281
2,Bayeux,La Closerie Teranga,9.1,Managed by a private host,49.278042,-0.701877
3,Amiens,WHITE HOUSE DHAVERNAS,8.6,Managed by a private host,49.881470,2.299871
4,Le Havre,Le Petit Vatel,7.1,"Le Petit Vatel is located in l’espace Perret, ...",49.491117,0.104518


In [12]:
# Merge our two datasets
df = df_hotels.merge(df, how='left', on='city')
df.head()

,city,hotel_name,score,description,latitude,longitude,temperature,humidity
0,Le Havre,Duplex lumineux tout confort,8.0,Duplex lumineux tout confort in Le Havre offer...,49.486276,0.131625,5,80
1,Le Havre,Les gîtes Havrais Bis,7.1,Located within 1.2 km of Le Havre Beach and 1....,49.504398,0.100281,5,80
2,Bayeux,La Closerie Teranga,9.1,Managed by a private host,49.278042,-0.701877,5,81
3,Amiens,WHITE HOUSE DHAVERNAS,8.6,Managed by a private host,49.881470,2.299871,2,87
4,Le Havre,Le Petit Vatel,7.1,"Le Petit Vatel is located in l’espace Perret, ...",49.491117,0.104518,5,80


In [13]:
# Create CSV file from dataframe
df.to_csv('hotel-weather.csv', encoding='utf-8', index=False)

In [14]:
# Import boto 3 and instanciate the session class
import boto3

session = boto3.Session()

In [18]:
# Fix axs S3 as resource
s3 = boto3.resource(
    service_name='s3',
    region_name='eu-west-3',
    aws_access_key_id='***',
    aws_secret_access_key='***'
)

In [19]:
# Upload files to S3 bucket
s3.Bucket('kayak-bucket-aws').upload_file(Filename='hotel-weather.csv', Key='hotel-weather.csv')

In [20]:
# Download csv file from S3 bucket
s3.Bucket('kayak-bucket-aws').download_file(Key='hotel-weather.csv', Filename='hotel-weather.csv')

In [21]:
# Define dataframe from S3 csv
df = pd.read_csv('hotel-weather.csv')

## Store all the information above in a data lake

In [22]:
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker 

In [23]:
# Define database connection variables
DBHOST = "database-kayak-nd.co0ylkdu9fwa.eu-west-3.rds.amazonaws.com"
DBUSER = "******"
DBPASS = "******"
DBNAME = "postgres"

In [49]:
# Import psycopg2 and connect to my postgresSQL database
import psycopg2
engine = create_engine(f"postgresql+psycopg2://{DBUSER}:{DBPASS}@{DBHOST}/{DBNAME}", pool_pre_ping=True)

In [32]:
# Initialize a sessionmaker to talk to our database
Session = sessionmaker(bind = engine)
# Instanciate session 
session = Session()

In [ ]:
# Transform the dataframes to SQL
df.to_sql("df_sql", engine)

In [34]:
# Create a statement
# SQL base selection for the next data analysis team
Selection = text("SELECT * FROM df_sql")
df_hotels = pd.read_sql(Selection, engine)

## Extract, transform and load cleaned data from your datalake to a data warehouse

#### Filter Data to find :
- best 5 cities based on temperature
- Top 20 hotels in our 5 best cities

In [37]:
# Create dataframe with descending value from temperature column
df_weather = df.groupby(by='city', as_index = False).sort_values(by = 'temperature',ascending = False)

In [38]:
# Create new dataframe with only five first rows
df_weather_top5 = df_weather.head(5)

In [39]:
# Create list of city with higher average temperature for the 7 days
Top5_cities = df_weather_top5.city.tolist()

In [40]:
# Create dataframe from the top5 list of best cities
df_hotels_best = df_hotels[df_hotels['city'].isin(Top5_cities)]

In [41]:
# Create dataframe from previous sorting values from city and score columns
df_hotels_best_score = df_hotels_best.sort_values(by = ['city','score'],ascending = False)

In [42]:
# Create grouped dataframe from previous by the city column
grouped_city_df = df_hotels_best_score.groupby('city')

In [43]:
# Create dataframe with only 20 first rows from preivou grouped dataframe
Top20_hotels = grouped_city_df.head(20)

## MAPBOX

In [45]:
import plotly.express as px

# Display weather map
fig = px.scatter_mapbox(df,
                        title='Weather map',
                        lat = 'latitude',
                        lon = 'longitude',
                        color = 'temperature',
                        color_continuous_scale = px.colors.diverging.balance,
                        size = 'humidity',
                        size_max = 30,
                        hover_name = 'city',
                        )
fig.update_layout(width = 1000,
                  height = 800,
                  title_x = 0.5, 
                  template = 'plotly_dark',
                  mapbox = {"style": "carto-darkmatter", "center": {"lon": 2, "lat" : 47}, "zoom": 4.8},
                  margin = {"l": 0, "r": 0, "b": 0, "t": 80},
                  )

fig.show()

In [48]:
# Display Top 20 hotels for the top 5 cities
fig = px.scatter_mapbox(Top20_hotels,
                        title = 'Best Hotels',
                        lat = 'latitude',
                        lon = 'longitude',
                        color = 'score',
                        size = 'score',
                        color_continuous_scale = px.colors.diverging.Tropic,
                        size_max = 20,
                        hover_name = 'hotel_name',
                        hover_data = {'description': True}
                        )
fig.update_layout(width = 1000,
                  height = 800,
                  title_x = 0.5, 
                  template='plotly_dark',
                  mapbox = {"style": "carto-darkmatter", "center": {"lon": 2, "lat" : 47}, "zoom": 4.8},
                  margin = {"l": 0, "r": 0, "b": 0, "t": 80},
                  )
fig.show()